In [1]:
import wntr

import matplotlib.pyplot as plt
import wntr.graphics.network as wntr_graphics

import pandas as pd
import numpy as np

import copy

In [2]:
import math

def calculate_distance(coord1, coord2):
    """Calculate Euclidean distance between two coordinates."""
    if coord1 is None or coord2 is None:
        return None
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

def civic_validator(wn, pipe_length_tolerance=1.0):
    """
    Comprehensive network validation checking for:
    - Phantom nodes and misregistrations
    - Coordinate issues (missing, at 0,0)
    - Pipe length vs geometric distance consistency
    
    Parameters:
    - wn: Water network model
    - pipe_length_tolerance: Acceptable difference in meters for pipe length validation
    """
    print("🔍 Running Sockford Civic Validator...\n")

    phantom_nodes = []
    undefined_links = []
    zero_coord_nodes = []
    missing_coord_nodes = []
    suspicious_coords = []
    length_mismatches = []
    no_coords_pipes = []

    # Check for nodes at (0,0) and nodes missing coordinates
    for name, node in wn.nodes():
        if hasattr(node, 'coordinates'):
            if node.coordinates == (0, 0):
                zero_coord_nodes.append(name)
            elif node.coordinates is None:
                missing_coord_nodes.append(name)
        else:
            missing_coord_nodes.append(name)

    # Check for links referencing undefined nodes
    for link_name, link in wn.links():
        if link.start_node_name not in wn.node_name_list or link.end_node_name not in wn.node_name_list:
            undefined_links.append(link_name)

    # Check for nodes not in junctions, tanks, or reservoirs
    all_nodes = set(wn.node_name_list)
    declared_nodes = set(wn.junction_name_list + wn.tank_name_list + wn.reservoir_name_list)
    phantom_nodes = list(all_nodes - declared_nodes)

    # Validate pipe lengths against geometric distances
    for link_name, link in wn.links():
        if hasattr(link, 'length'):  # It's a pipe
            start_node = wn.get_node(link.start_node_name)
            end_node = wn.get_node(link.end_node_name)
            
            # Check if coordinates exist
            if not hasattr(start_node, 'coordinates') or not hasattr(end_node, 'coordinates'):
                no_coords_pipes.append(link_name)
                continue
            
            if start_node.coordinates is None or end_node.coordinates is None:
                no_coords_pipes.append(link_name)
                continue
            
            # Calculate geometric distance
            calc_dist = calculate_distance(start_node.coordinates, end_node.coordinates)
            
            if calc_dist is not None:
                diff = abs(link.length - calc_dist)
                
                # Check for suspiciously small geometric distances
                if calc_dist < 1.0 and link.length > 10.0:
                    suspicious_coords.append({
                        'pipe': link_name,
                        'start': link.start_node_name,
                        'end': link.end_node_name,
                        'start_coords': start_node.coordinates,
                        'end_coords': end_node.coordinates,
                        'pipe_length': link.length,
                        'calc_distance': calc_dist
                    })
                
                # Check for length mismatches
                elif diff > pipe_length_tolerance:
                    length_mismatches.append({
                        'pipe': link_name,
                        'start': link.start_node_name,
                        'end': link.end_node_name,
                        'pipe_length': link.length,
                        'calc_distance': calc_dist,
                        'difference': diff,
                        'ratio': link.length / calc_dist if calc_dist > 0 else float('inf')
                    })

    # Report all findings
    print("=" * 80)
    print("NODE VALIDATION")
    print("=" * 80)
    
    if zero_coord_nodes:
        print(f"⚠️  Nodes at (0,0): {zero_coord_nodes}")
    if missing_coord_nodes:
        print(f"⚠️  Nodes missing coordinates: {missing_coord_nodes}")
    if undefined_links:
        print(f"⚠️  Links referencing undefined nodes: {undefined_links}")
    if phantom_nodes:
        print(f"⚠️  Phantom nodes (undeclared): {phantom_nodes}")
    if not (zero_coord_nodes or undefined_links or phantom_nodes or missing_coord_nodes):
        print("✅ No node issues detected.")
    
    print("\n" + "=" * 80)
    print("PIPE LENGTH VALIDATION")
    print("=" * 80)
    
    total_pipes = len([l for l in wn.links() if hasattr(l[1], 'length')])
    print(f"Total pipes checked: {total_pipes}\n")
    
    if suspicious_coords:
        print(f"⚠️  SUSPICIOUS COORDINATES ({len(suspicious_coords)} pipes):")
        print("    These pipes have large declared lengths but tiny geometric distances,")
        print("    suggesting coordinate problems (possibly at 0,0):\n")
        for item in suspicious_coords[:10]:  # Show first 10
            print(f"    Pipe {item['pipe']}: {item['start']} -> {item['end']}")
            print(f"      Start coords: {item['start_coords']}")
            print(f"      End coords:   {item['end_coords']}")
            print(f"      Pipe length:  {item['pipe_length']:.2f} m")
            print(f"      Calc dist:    {item['calc_distance']:.2f} m")
            print()
        if len(suspicious_coords) > 10:
            print(f"    ... and {len(suspicious_coords) - 10} more")
    
    if length_mismatches:
        print(f"\n⚠️  LENGTH MISMATCHES ({len(length_mismatches)} pipes):")
        for item in length_mismatches[:10]:  # Show first 10
            print(f"    Pipe {item['pipe']}: Declared={item['pipe_length']:.2f}m, " +
                  f"Calculated={item['calc_distance']:.2f}m, Diff={item['difference']:.2f}m")
        if len(length_mismatches) > 10:
            print(f"    ... and {len(length_mismatches) - 10} more")
    
    if no_coords_pipes:
        print(f"\n⚠️  PIPES WITH MISSING COORDINATES ({len(no_coords_pipes)} pipes):")
        print(f"    {no_coords_pipes[:10]}")
        if len(no_coords_pipes) > 10:
            print(f"    ... and {len(no_coords_pipes) - 10} more")
    
    if not (suspicious_coords or length_mismatches or no_coords_pipes):
        print("✅ All pipe lengths match geometric distances!")

    print("\n" + "=" * 80)
    print("🧾 Civic audit complete.")
    print("=" * 80)
    
    # Return summary for programmatic use
    return {
        'zero_coord_nodes': zero_coord_nodes,
        'missing_coord_nodes': missing_coord_nodes,
        'undefined_links': undefined_links,
        'phantom_nodes': phantom_nodes,
        'suspicious_coords': suspicious_coords,
        'length_mismatches': length_mismatches,
        'no_coords_pipes': no_coords_pipes,
        'total_issues': (len(zero_coord_nodes) + len(missing_coord_nodes) + 
                        len(undefined_links) + len(phantom_nodes) + 
                        len(suspicious_coords) + len(length_mismatches) + 
                        len(no_coords_pipes))
    }

In [3]:
# Load network

# works on my system
#wn = wntr.network.WaterNetworkModel('data/Net1_EPANET-EXAMPLE_1s.inp') 
#wn = wntr.network.WaterNetworkModel('data/Net3_EPANET-EXAMPLE_No_Demand_Change.inp')
#wn = wntr.network.WaterNetworkModel('data/Net3_EPANET-EXAMPLE.inp')

# does not work on my system
 
#wn = wntr.network.WaterNetworkModel('data/Micropolis_TEVA-SPOT_Adjusted_PumpCurve3&4.inp')
wn = wntr.network.WaterNetworkModel('data/Net3_(BWSN-2)_Morph_Error_Free_1s-WQ.inp')

# Run comprehensive validation
results = civic_validator(wn)

c:\robots\water\venv\lib\site-packages\wntr\epanet\io.py:2082: UserWarning: Not all curves were used in "data/Net3_(BWSN-2)_Morph_Error_Free_1s-WQ.inp"; added with type None, units conversion left to user
  warnings.warn('Not all curves were used in "{}"; added with type None, units conversion left to user'.format(self.wn.name))


🔍 Running Sockford Civic Validator...

NODE VALIDATION
✅ No node issues detected.

PIPE LENGTH VALIDATION
Total pipes checked: 14822


⚠️  LENGTH MISMATCHES (14819 pipes):
    Pipe LINK-0: Declared=73.82m, Calculated=242.14m, Diff=168.32m
    Pipe LINK-1: Declared=188.49m, Calculated=892.36m, Diff=703.87m
    Pipe LINK-10: Declared=98.51m, Calculated=482.10m, Diff=383.59m
    Pipe LINK-100: Declared=83.55m, Calculated=357.61m, Diff=274.07m
    Pipe LINK-1000: Declared=38.19m, Calculated=187.97m, Diff=149.78m
    Pipe LINK-10000: Declared=81.41m, Calculated=388.02m, Diff=306.61m
    Pipe LINK-10001: Declared=75.50m, Calculated=361.17m, Diff=285.67m
    Pipe LINK-10002: Declared=135.45m, Calculated=618.66m, Diff=483.20m
    Pipe LINK-10003: Declared=178.22m, Calculated=618.61m, Diff=440.39m
    Pipe LINK-10004: Declared=90.65m, Calculated=420.08m, Diff=329.44m
    ... and 14809 more

🧾 Civic audit complete.
NODE VALIDATION
✅ No node issues detected.

PIPE LENGTH VALIDATION
Total pipes ch

In [4]:
# Analyze which is more trustworthy: declared lengths or coordinates
def analyze_length_reliability(results):
    """
    Analyze the pattern of mismatches to determine if coordinates or 
    declared lengths are more reliable.
    """
    print("🔬 RELIABILITY ANALYSIS")
    print("=" * 80)
    
    if not results['length_mismatches']:
        print("No length mismatches to analyze.")
        return
    
    # Calculate statistics
    ratios = [m['ratio'] for m in results['length_mismatches']]
    differences = [m['difference'] for m in results['length_mismatches']]
    
    import numpy as np
    
    print(f"Total mismatches: {len(ratios)}")
    print(f"\nRatio statistics (Declared/Calculated):")
    print(f"  Mean ratio: {np.mean(ratios):.3f}")
    print(f"  Median ratio: {np.median(ratios):.3f}")
    print(f"  Std dev: {np.std(ratios):.3f}")
    print(f"  Min ratio: {np.min(ratios):.3f}")
    print(f"  Max ratio: {np.max(ratios):.3f}")
    
    print(f"\nDifference statistics (meters):")
    print(f"  Mean difference: {np.mean(differences):.2f} m")
    print(f"  Median difference: {np.median(differences):.2f} m")
    print(f"  Max difference: {np.max(differences):.2f} m")
    
    # Determine recommendation
    print(f"\n💡 RECOMMENDATION:")
    
    # If suspicious coords exist, coordinates are likely wrong
    if results['suspicious_coords']:
        print(f"  ⚠️  Found {len(results['suspicious_coords'])} pipes with suspicious coordinates")
        print(f"      (huge declared length, tiny calculated distance)")
        print(f"  → This suggests COORDINATES may be unreliable (some nodes at wrong positions)")
        print(f"  → SAFER: Keep declared pipe lengths, fix node coordinates")
    
    # If ratios are consistently in one direction
    elif np.median(ratios) > 1.5 or np.median(ratios) < 0.67:
        print(f"  ⚠️  Ratios are skewed (median: {np.median(ratios):.2f})")
        print(f"  → Suggests systematic scaling issue")
        if np.median(ratios) > 1.5:
            print(f"  → Declared lengths consistently LARGER than calculated")
        else:
            print(f"  → Declared lengths consistently SMALLER than calculated")
        print(f"  → Need to investigate coordinate system or units")
    
    # If differences are small and random
    elif np.median(differences) < 10:
        print(f"  ✅ Differences are relatively small (median: {np.median(differences):.2f}m)")
        print(f"  → Could use either, but coordinates might be more precise")
        print(f"  → SAFER: Update pipe lengths from coordinates (if coordinates look good)")
    
    else:
        print(f"  ⚠️  Large differences detected")
        print(f"  → Manual review recommended before making changes")

if 'results' in locals():
    analyze_length_reliability(results)

🔬 RELIABILITY ANALYSIS
Total mismatches: 14819

Ratio statistics (Declared/Calculated):
  Mean ratio: 0.281
  Median ratio: 0.275
  Std dev: 0.217
  Min ratio: 0.004
  Max ratio: 7.810

Difference statistics (meters):
  Mean difference: 357.32 m
  Median difference: 284.25 m
  Max difference: 11223.69 m

💡 RECOMMENDATION:
  ⚠️  Ratios are skewed (median: 0.28)
  → Suggests systematic scaling issue
  → Declared lengths consistently SMALLER than calculated
  → Need to investigate coordinate system or units


In [5]:
# Function to update pipe lengths from coordinates
def fix_pipe_lengths_from_coordinates(wn, min_length=0.1):
    """
    Update all pipe lengths to match geometric distances calculated from node coordinates.
    
    Parameters:
    - wn: Water network model (will be modified in place)
    - min_length: Minimum pipe length to enforce (meters)
    
    Returns:
    - Dictionary with update statistics
    """
    print("🔧 Updating pipe lengths from coordinates...\n")
    
    updated = []
    skipped_no_coords = []
    skipped_too_short = []
    
    for link_name, link in wn.links():
        if hasattr(link, 'length'):  # It's a pipe
            start_node = wn.get_node(link.start_node_name)
            end_node = wn.get_node(link.end_node_name)
            
            # Check if coordinates exist
            if (not hasattr(start_node, 'coordinates') or 
                not hasattr(end_node, 'coordinates') or
                start_node.coordinates is None or 
                end_node.coordinates is None):
                skipped_no_coords.append(link_name)
                continue
            
            # Calculate new length
            old_length = link.length
            new_length = calculate_distance(start_node.coordinates, end_node.coordinates)
            
            if new_length is not None:
                # Enforce minimum length
                if new_length < min_length:
                    skipped_too_short.append({
                        'pipe': link_name,
                        'calculated_length': new_length,
                        'enforced_length': min_length
                    })
                    new_length = min_length
                
                # Update the pipe length
                if abs(old_length - new_length) > 0.01:  # Only if changed
                    link.length = new_length
                    updated.append({
                        'pipe': link_name,
                        'old_length': old_length,
                        'new_length': new_length,
                        'change': new_length - old_length
                    })
    
    # Report results
    print(f"✅ Updated {len(updated)} pipe lengths")
    print(f"⚠️  Skipped {len(skipped_no_coords)} pipes (missing coordinates)")
    print(f"⚠️  Enforced minimum length on {len(skipped_too_short)} pipes")
    
    if updated:
        print(f"\nSample updates:")
        for item in updated[:5]:
            print(f"  {item['pipe']}: {item['old_length']:.2f}m → {item['new_length']:.2f}m " +
                  f"(change: {item['change']:+.2f}m)")
        if len(updated) > 5:
            print(f"  ... and {len(updated) - 5} more")
    
    return {
        'updated': updated,
        'skipped_no_coords': skipped_no_coords,
        'skipped_too_short': skipped_too_short,
        'num_updated': len(updated)
    }

# Example usage (commented out - uncomment to actually modify the network):
fix_results = fix_pipe_lengths_from_coordinates(wn)

# Re-validate after fixing
print("\n" + "="*80)
print("RE-VALIDATION AFTER FIXES")
print("="*80 + "\n")
results_after = civic_validator(wn)

print("Function ready. Uncomment the code above to fix pipe lengths.")

🔧 Updating pipe lengths from coordinates...

✅ Updated 14822 pipe lengths
⚠️  Skipped 0 pipes (missing coordinates)
⚠️  Enforced minimum length on 0 pipes

Sample updates:
  LINK-0: 73.82m → 242.14m (change: +168.32m)
  LINK-1: 188.49m → 892.36m (change: +703.87m)
  LINK-10: 98.51m → 482.10m (change: +383.59m)
  LINK-100: 83.55m → 357.61m (change: +274.07m)
  LINK-1000: 38.19m → 187.97m (change: +149.78m)
  ... and 14817 more

RE-VALIDATION AFTER FIXES

🔍 Running Sockford Civic Validator...

✅ Updated 14822 pipe lengths
⚠️  Skipped 0 pipes (missing coordinates)
⚠️  Enforced minimum length on 0 pipes

Sample updates:
  LINK-0: 73.82m → 242.14m (change: +168.32m)
  LINK-1: 188.49m → 892.36m (change: +703.87m)
  LINK-10: 98.51m → 482.10m (change: +383.59m)
  LINK-100: 83.55m → 357.61m (change: +274.07m)
  LINK-1000: 38.19m → 187.97m (change: +149.78m)
  ... and 14817 more

RE-VALIDATION AFTER FIXES

🔍 Running Sockford Civic Validator...

NODE VALIDATION
✅ No node issues detected.

PIPE LE

In [7]:
# Save the revised network to a new INP file
import os

# Determine the original filename
original_file = 'data/Net3_(BWSN-2)_Morph_Error_Free_1s-WQ.inp'

# Create the revised filename
directory = os.path.dirname(original_file)
filename = os.path.basename(original_file)
revised_filename = os.path.join(directory, f"revised_{filename}")

# Save the network - WNTR uses write_inpfile method
print(f"💾 Saving revised network to: {revised_filename}")

try:
    # Try the standard WNTR method
    wn.write_inpfile(revised_filename)
    print(f"✅ Network saved successfully using write_inpfile()!")
except AttributeError:
    # If that doesn't work, try alternative method
    try:
        from wntr.epanet.io import InpFile
        InpFile.write(revised_filename, wn)
        print(f"✅ Network saved successfully using InpFile.write()!")
    except Exception as e:
        print(f"❌ Error saving file: {e}")
        print(f"   Trying manual approach...")
        
        # Last resort: use the to_dict and recreate
        print(f"   Please check WNTR documentation for the correct write method")

# Verify the file was created
if os.path.exists(revised_filename):
    file_size = os.path.getsize(revised_filename)
    print(f"   File size: {file_size:,} bytes")
    print(f"   Location: {os.path.abspath(revised_filename)}")
else:
    print("⚠️  Warning: File was not created!")

💾 Saving revised network to: data\revised_Net3_(BWSN-2)_Morph_Error_Free_1s-WQ.inp
❌ Error saving file: InpFile.write() missing 1 required positional argument: 'wn'
   Trying manual approach...
   Please check WNTR documentation for the correct write method
⚠️  Warning: File was not created!
